In [315]:
import numpy as np
import pandas as pd
import os
import time, calendar, datetime
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
import glob
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from sklearn import preprocessing
from sklearn.cluster import KMeans
from collections import Counter
import plotly.express as px

Populating the interactive namespace from numpy and matplotlib


In [316]:
!pip install markovify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [317]:
!pip install modules


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [318]:
df = pd.read_csv('/content/data (2).csv')



In [319]:
df[:5]

,lat,long,Timestamp
0,19.001965,73.055221,3/23/2023 22:26
1,19.076743,72.951533,2/7/2023 17:59
2,19.328718,72.783555,3/3/2023 5:32
3,19.523080,73.200119,1/26/2023 19:32
4,19.354632,72.418512,1/9/2023 20:50


In [320]:
intension = ['Personal', 'Family', 'Event', 'Organisation']
l=[]
for i in range(0, len(df['lat'])):
  x = random.choice(intension)
  l.append(x)


In [321]:
df['Event']=l
l


['Organisation',
 'Organisation',
 'Event',
 'Event',
 'Organisation',
 'Event',
 'Event',
 'Event',
 'Personal',
 'Event',
 'Organisation',
 'Event',
 'Personal',
 'Family',
 'Organisation',
 'Personal',
 'Family',
 'Family',
 'Personal',
 'Organisation',
 'Organisation',
 'Family',
 'Event',
 'Organisation',
 'Organisation',
 'Personal',
 'Personal',
 'Event',
 'Personal',
 'Family',
 'Event',
 'Event',
 'Organisation',
 'Personal',
 'Event',
 'Family',
 'Family',
 'Event',
 'Event',
 'Family',
 'Family',
 'Family',
 'Event',
 'Family',
 'Organisation',
 'Personal',
 'Personal',
 'Personal',
 'Personal',
 'Organisation',
 'Organisation',
 'Organisation',
 'Personal',
 'Organisation',
 'Organisation',
 'Family',
 'Family',
 'Organisation',
 'Family',
 'Organisation',
 'Family',
 'Personal',
 'Event',
 'Organisation',
 'Family',
 'Personal',
 'Organisation',
 'Organisation',
 'Personal',
 'Family',
 'Family',
 'Family',
 'Event',
 'Family',
 'Family',
 'Organisation',
 'Organisation',


In [322]:
df.head()

,lat,long,Timestamp,Event
0,19.001965,73.055221,3/23/2023 22:26,Organisation
1,19.076743,72.951533,2/7/2023 17:59,Organisation
2,19.328718,72.783555,3/3/2023 5:32,Event
3,19.523080,73.200119,1/26/2023 19:32,Event
4,19.354632,72.418512,1/9/2023 20:50,Organisation


In [323]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [324]:

# Extract latitude and longitude coordinates from the dataframe and store as numpy array
coords = df[['lat', 'long']].values 

# Set the radius of the Earth in kilometers (for use in haversine distance calculation)
kms_rad = 6371.088

# Set the maximum distance between two points to be considered part of the same cluster
# In this case, it is set to 50 kilometers, converted to radians using the Earth's radius
epsilon = 100/ kms_rad

# Perform DBSCAN clustering on the latitude and longitude coordinates using haversine distance metric
# Require at least 50 points in a cluster to be considered a cluster
db = DBSCAN(eps=epsilon, min_samples=5, metric="haversine").fit(np.radians(coords))

# Retrieve the cluster labels assigned by DBSCAN
cluster_labels = db.labels_

# Count the number of clusters found (excluding noise points with label -1)
num_clusters = len(set(cluster_labels)) - 1

# Print the total number of points and the number of clusters found
print('Clustered ' + str(len(df)) + ' points to ' + str(num_clusters) + ' clusters.')


Clustered 100 points to 12 clusters.


In [325]:

df['clusters'] = cluster_labels
cluster_labels

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,
        5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  0,  0,  0,
        0,  0,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  5,  5,  5,  5,  5,
       10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12])

In [326]:
a=(df['clusters'].unique().shape[0])
a

13

In [327]:
px.scatter(df, x="long", y="lat", color="clusters")

In [328]:
df['clusters'].value_counts().to_frame()

,clusters
0,15
1,10
2,10
3,10
4,10
5,10
6,5
7,5
8,5
9,5


In [329]:
temp = df['clusters']
len(np.array(temp))


100

In [330]:
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [331]:
! pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [332]:
# Import necessary libraries
from shapely.geometry import MultiPoint
from geopy.distance import great_circle

# Define a function to get the centermost point of a cluster
def get_centremost_point(cluster):
    # Calculate the centroid of the cluster
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    
    # Find the point in the cluster that is closest to the centroid
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    
    # Return the coordinates of the centermost point
    return tuple(centermost_point)


In [333]:
 clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])


In [334]:
# Apply the get_centremost_point function to each cluster and extract the coordinates of the centermost point
centermost_points = clusters.map(get_centremost_point)

# Unpack the coordinates into separate latitude and longitude tuples
lats, longs = zip(*centermost_points)

# Create a new DataFrame to hold the representative points
rep_points = pd.DataFrame({"lat": lats, "long": longs})

# Print the representative points DataFrame
print(rep_points)


          lat       long
0   18.839839  73.063121
1   22.492147  88.471564
2   28.679740  77.486963
3   17.051449  78.186149
4   12.972557  77.655264
5   23.748818  86.724683
6   20.157919  85.957497
7   26.457589  83.284346
8   26.233335  80.058392
9   26.414168  85.481511
10  33.858731  74.838291
11  26.910355  75.955728


In [335]:
centroid_list = []
for i in range(num_clusters):
    centroid = (rep_points['lat'][i], rep_points['long'][i])
    centroid_list.append(centroid)

centroid_list

[(18.83983908, 73.06312133),
 (22.49214734, 88.47156363),
 (28.67974026, 77.48696253),
 (17.05144945, 78.18614926),
 (12.97255671, 77.65526403),
 (23.7488184, 86.72468279),
 (20.1579193, 85.95749729),
 (26.45758931, 83.28434576),
 (26.23333464, 80.05839246),
 (26.41416753, 85.48151088),
 (33.85873103, 74.83829137),
 (26.91035498, 75.95572775)]

In [336]:
import folium
from haversine import haversine

# Define the center of the map and the initial zoom level
center = [20.5937, 78.9629]
zoom = 4

# Create the map object
india_map = folium.Map(location=center, zoom_start=zoom)

# Create a marker for the first location in centroid_list
folium.Marker(location=[centroid_list[0][0], centroid_list[0][1]],popup='present Location').add_to(india_map)

# Loop through each location in centroid_list
for i in range(1, len(centroid_list)):
    
    # Calculate the distance between this location and the previous one
    prev_location = (float(centroid_list[i-1][0]), float(centroid_list[i-1][1]))
    curr_location = (float(centroid_list[i][0]), float(centroid_list[i][1]))
    distance = haversine(prev_location, curr_location)
    
    # Create a tooltip text for the line connecting the locations
    tooltip_text = f"Distance from previous location: {distance:.2f} km"
    
    # Create a polyline connecting the previous location to this location
    line_coords = [prev_location, curr_location]
    folium.PolyLine(locations=line_coords, color='blue', tooltip=tooltip_text).add_to(india_map)
    
    # Create a green marker for this location with the tooltip text
    folium.Marker(location=[centroid_list[i][0], centroid_list[i][1]],
                  icon=folium.Icon(color='green'),
                  popup=tooltip_text).add_to(india_map)

# Display the map
india_map


In [337]:
centroid_list


[(18.83983908, 73.06312133),
 (22.49214734, 88.47156363),
 (28.67974026, 77.48696253),
 (17.05144945, 78.18614926),
 (12.97255671, 77.65526403),
 (23.7488184, 86.72468279),
 (20.1579193, 85.95749729),
 (26.45758931, 83.28434576),
 (26.23333464, 80.05839246),
 (26.41416753, 85.48151088),
 (33.85873103, 74.83829137),
 (26.91035498, 75.95572775)]

In [360]:
centroids= np.array(centroid_list)
index = int(input("Enter the cluster number"))
# for i in centroid_list:
  # index = np.where(centroids == ((i[0], i[1])))[0][0]
centroidnearplaces=df.loc[(df['clusters'] == index)]
preloc=[]
preloc =centroidnearplaces.loc[:, ['lat', 'long',  'Event','clusters']]
columname=['lat','long','Event','clusters']
preloc = np.array(preloc)
# dfc=pd.DataFrame(data=preloc,columns=columname)
# print(dfc)

Enter the cluster number4


In [361]:
preloc=[]

preloc =centroidnearplaces.loc[:, ['lat', 'long',  'Event']]
preloc = np.array(preloc)

In [362]:
preloc

array([[13.25409919, 77.39449623, 'Family'],
       [12.5788081, 77.37372929, 'Family'],
       [13.41738998, 77.1393437, 'Event'],
       [12.53900235, 77.26922457, 'Family'],
       [13.24513588, 77.9388109, 'Organisation'],
       [12.97255671, 77.65526403, 'Personal'],
       [12.87879807, 77.39427144, 'Personal'],
       [12.90632081, 77.8350108, 'Personal'],
       [13.29669422, 77.32554609, 'Personal'],
       [12.83382439, 77.9099768, 'Organisation']], dtype=object)

In [341]:
event_list=[]
event_list=centroidnearplaces.loc[:, ['Event']]
event_list


,Event
90,Family
91,Family
92,Event
93,Personal
94,Organisation


In [342]:
unique_count= centroidnearplaces['Event'].nunique()
v=100/unique_count
li=[]
n=len(event_list)
li.extend([v] * n)
li

[25.0, 25.0, 25.0, 25.0, 25.0]

In [343]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [363]:
#@title Default title text
import folium
from haversine import haversine


# Define the center of the map and the initial zoom level
center = [20.5937, 78.9629]
zoom = 4

# Create the map object
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=zoom)
for i in preloc:
    # access the latitude and longitude values for each row
    
    folium.Marker(location=[i[0],i[1] ],icon=folium.Icon(color='green')).add_to(india_map)
# Display the map
india_map



In [345]:
latlong=np.array(df[['lat','long']])

In [346]:
uniplaces=np.unique(latlong, axis = 0)

In [348]:
prob = np.zeros((len(uniplaces), len(uniplaces)), dtype = int)

In [358]:
for i in range(0, len(latlong)-1):
  num1 = np.where(uniplaces == [latlong[i]])[0][0]
  num2 = np.where(uniplaces ==  latlong[i+1])[0][0]
  prob[num1][num2] += 1
prob


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
temp = [np.sum(prob[i]) for i in range(0, len(prob))]
#temp

In [352]:
ans = [np.where(prob[i] != 0)[0] for i in range(len(prob))]

In [353]:
for i in range(0, len(prob)):
  prob[i][ans[i]] = prob[i][ans[i]]/temp[i] 

prob

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [373]:
def predict(date):
   predictedplaces=[]
   initial_index = np.where(df.Timestamp ==date)[0][0]
   place = prob[initial_index]
   for i in range(1, len(place)):
     if place[i] != 0:
      prediction_index = latlong[i+1]
      predictedplaces.append(prediction_index)
   return predictedplaces

In [376]:
li=predict(df.Timestamp[1])
print(df.Timestamp[1])

2/7/2023 17:59


In [377]:
li

[array([22.44166458, 88.69885555])]

In [378]:
import folium
from haversine import haversine

# Define the center of the map and the initial zoom level
center = [20.5937, 78.9629]
zoom = 4
india = folium.Map(location=[20.5937, 78.9629], zoom_start=zoom)
folium.Marker(location=[li[0][0], li[0][1]]).add_to(india)

# Loop through each location in centroid_list
for i in range(1, len(li)):
    
    # Calculate the distance between this location and the previous one
    prev_location = (float(li[i-1][0]), float(li[i-1][1]))
    curr_location = (float(li[i][0]), float(li[i][1]))
    distance = haversine(prev_location, curr_location)
    
    # Create a tooltip text for the line connecting the locations
    tooltip_text = f"Distance from previous location: {distance:.2f} km"
    
    # Create a polyline connecting the previous location to this location
    line_coords = [prev_location, curr_location]
    folium.PolyLine(locations=line_coords, color='blue', tooltip=tooltip_text).add_to(india)
    
    # Create a green marker for this location with the tooltip text
    folium.Marker(location=[li[i][0], li[i][1]],
                  icon=folium.Icon(color='green'),
                  popup=tooltip_text).add_to(india)

# Display the map
india